# 1. Import Dependencies

In [2]:
!pip install stable-baselines3[extra] -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 7.9 MB/s eta 0:00:00


In [8]:
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import os
import ale_py

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# 2. Test Environment

In [4]:
environment_name = "Breakout-v0"

In [9]:
env = make_atari_env(environment_name, n_envs=1)

/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment Breakout-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


In [13]:
episodes = 5
for episode in range(1, episodes+1):
    # Based on the ValueError, env.reset() returns a single value (the batch observation).
    # The 'info' dictionary is not directly returned here for this specific VecEnv configuration.
    obs_vec = env.reset() # Capture the single returned batch observation
    state = obs_vec[0] # Extract observation for the single environment from the batch
    done = False
    score = 0

    while not done:
        env.render()
        action_scalar = env.action_space.sample() # Sample a scalar action
        actions_for_env_step = [action_scalar] # Wrap it in a list for VecEnv.step

        # env.step returns arrays/lists for n_envs=1
        n_state_vec, reward_vec, done_vec, info_list_step = env.step(actions_for_env_step)

        # Extract the scalar values for the single environment simulation
        n_state = n_state_vec[0]
        reward = reward_vec[0]
        done = done_vec[0]
        info = info_list_step[0]

        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:1.0
Episode:2 Score:0.0
Episode:3 Score:0.0
Episode:4 Score:0.0
Episode:5 Score:0.0


In [14]:
env.action_space.sample()

np.int64(3)

In [15]:
env.observation_space.sample()

array([[[  4],
        [ 46],
        [239],
        ...,
        [206],
        [160],
        [ 18]],

       [[234],
        [105],
        [ 22],
        ...,
        [161],
        [163],
        [137]],

       [[251],
        [ 30],
        [175],
        ...,
        [ 23],
        [230],
        [ 22]],

       ...,

       [[ 40],
        [  8],
        [ 63],
        ...,
        [190],
        [ 56],
        [ 84]],

       [[147],
        [ 30],
        [174],
        ...,
        [250],
        [ 69],
        [122]],

       [[237],
        [ 16],
        [115],
        ...,
        [  1],
        [227],
        [189]]], dtype=uint8)

# 3. Vectorise Environment and Train Model

In [16]:
env = make_atari_env('Breakout-v0', n_envs=4, seed=0)

/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment Breakout-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


In [17]:
env = VecFrameStack(env, n_stack=4)

In [18]:
log_path = os.path.join('Training', 'Logs')

In [19]:
model = A2C("CnnPolicy", env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env in a VecTransposeImage.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
model.learn(total_timesteps=400000)

# 4. Save and Reload Model

In [20]:
a2c_path = os.path.join('Training', 'Saved Models', 'A2C_model')

In [21]:
model.save(a2c_path)

/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'Training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [22]:
del model

In [23]:
env = make_atari_env('Breakout-v0', n_envs=1, seed=0)
env = VecFrameStack(env, n_stack=4)

/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment Breakout-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


In [24]:
model = A2C.load(a2c_path, env)

Wrapping the env in a VecTransposeImage.


# 5. Evaluate and Test

In [25]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(np.float64(1.3), np.float64(0.6403124237432849))

In [ ]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

In [27]:
env.close()